In [1]:
import numpy as np
from itertools import product

# quara
from quara.objects.composite_system_typical import generate_composite_system
from quara.objects.tester_typical import (
    generate_tester_states,
    generate_tester_povms,
)
from quara.objects.qoperation_typical import generate_qoperation
from quara.protocol.qtomography.standard.standard_qst import StandardQst
from quara.protocol.qtomography.standard.standard_povmt import StandardPovmt
from quara.protocol.qtomography.standard.standard_qpt import StandardQpt

from quara.interface.cvxpy.standard_qtomography.loss_function import (
    CvxpyLossFunction,
    CvxpyLossFunctionOption,
    CvxpyRelativeEntropy,
    CvxpyUniformSquaredError,
    CvxpyApproximateRelativeEntropyWithoutZeroProbabilityTerm,
    CvxpyApproximateRelativeEntropyWithZeroProbabilityTerm,
    CvxpyApproximateRelativeEntropyWithZeroProbabilityTermSquared,
)
from quara.protocol.qtomography.standard.preprocessing import (
    combine_nums_prob_dists,
)
from quara.interface.cvxpy.standard_qtomography.minimization_algorithm import (
    CvxpyMinimizationAlgorithm,
    CvxpyMinimizationAlgorithmOption,
)
from quara.interface.cvxpy.standard_qtomography.estimator import (
    CvxpyLossMinimizationEstimator,
)

In [2]:
mode = "qubit"
num = 1
c_sys = generate_composite_system(mode=mode, num=num)

# Testers
names = ["x", "y", "z"]
testers = generate_tester_povms(c_sys=c_sys, names = names)

seed = 7896
sqt = StandardQst(testers, on_para_eq_constraint=True, schedules="all", seed=seed)

mode = "state"
name = "z0"
true = generate_qoperation(mode=mode, name=name, c_sys=c_sys)

num_data = 1000
empi_dists = sqt.generate_empi_dists(state=true, num_sum=num_data)

prob_dists_true = sqt.calc_prob_dists(true)
nums = [num_data]*sqt.num_schedules
empi_dists = combine_nums_prob_dists(nums, prob_dists_true)

In [3]:
num_sums = [100, 1000]
empi_dists_sequence = sqt.generate_empi_dists_sequence(true, num_sums=num_sums)

In [4]:
#loss = CvxpyRelativeEntropy()
#loss = CvxpyUniformSquaredError()
#loss = CvxpyApproximateRelativeEntropyWithoutZeroProbabilityTerm()
#loss = CvxpyApproximateRelativeEntropyWithZeroProbabilityTerm()
loss = CvxpyApproximateRelativeEntropyWithZeroProbabilityTermSquared()
modes_form = ['sum', 'quadratic']
loss_options = [CvxpyLossFunctionOption(mode_form=mode_form) for mode_form in modes_form]
algo = CvxpyMinimizationAlgorithm()
estimator = CvxpyLossMinimizationEstimator()

In [6]:
modes_constraint = ["physical", "physical_and_zero_probability_equation_satisfied"]

for pro in product(loss_options, modes_constraint):
    loss_option = pro[0]
    mode_constraint = pro[1]
    algo_option = CvxpyMinimizationAlgorithmOption(name_solver="mosek", mode_constraint=mode_constraint)
    result = estimator.calc_estimate_sequence(
        qtomography=sqt, 
        empi_dists_sequence=empi_dists_sequence, 
        loss=loss, 
        loss_option=loss_option, 
        algo=algo, 
        algo_option=algo_option, 
        is_computation_time_required=True)
    print(result.estimated_var_sequence)
    print(result.estimated_loss_sequence)
    print(result.computation_times)

[array([0.02816296, 0.07040962, 0.70302823]), array([-0.01696045,  0.02261394,  0.70653756])]
[4.198086630931133e-06, 8.110866839464571e-08]
[0.07919692993164062, 0.05410146713256836]
[array([8.53967885e-06, 2.13537063e-05, 7.07106781e-01]), array([-1.88630058e-05,  2.51423634e-05,  7.07106781e-01])]
[0.0019321658620985586, 0.0002660743145577208]
[0.05899786949157715, 0.06303977966308594]
[array([0.02816196, 0.07040747, 0.70302894]), array([-0.01696034,  0.0226138 ,  0.70653875])]
[4.193151942027962e-06, 8.076049712313349e-08]
[0.04114484786987305, 0.03819704055786133]
[array([5.84545816e-06, 1.47379358e-05, 7.07106781e-01]), array([-1.92321104e-05,  2.56543902e-05,  7.07106781e-01])]
[0.001949783614819539, 0.00026629217567455954]
[0.04451894760131836, 0.03768610954284668]
